# Feed the results of the ESM back in the LCI database 
The construction and operation of technologies at year t as well as the utilization of resources at year t are allocated to year t. Thus, those shall not be included again in order to avoid double-counting. However, new constructions at year t+1 are using the mix build over the past iterations. This mix is defined by the shares resulting from the ESM at the previous iteration. As mentioned before, the construction phase should be set to zero as it was allocated to the construction year.  

In [1]:
import pandas as pd
import bw2data as bd
from mescal import *

In [2]:
bd.projects.set_current("ei3.8-mescal")

In [3]:
annual_prod = pd.read_csv('energyscope_data/results_ES.csv')
model = pd.read_csv('energyscope_data/model.csv')
mapping = pd.read_csv('energyscope_data/mapping_linked.csv')
unit_conversion = pd.read_csv('energyscope_data/unit_conversion.csv')
mapping_esm_flows_to_CPC = pd.read_csv('../dev/energyscope_data/mapping_esm_flows_to_CPC.csv')
technology_compositions = pd.read_csv('../dev/energyscope_data/technology_compositions.csv')
technology_specifics = pd.read_csv('../dev/energyscope_data/technology_specifics.csv')
new_end_use_types = pd.read_csv('../dev/energyscope_data/new_end_use_types.csv')
tech_to_remove_from_layers = pd.read_csv('../dev/energyscope_data/technologies_to_remove_from_layers.csv')

In [4]:
name_premise_with_CPC_db = 'ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_with_CPC'

In [5]:
premise_db_with_CPC = load_extract_db(name_premise_with_CPC_db)

In [6]:
db = concatenate_databases(list(mapping.Database.unique()))

In [7]:
# Define the user-defined ranking
my_ranking = [
    'CA-QC',  # Quebec
    'CA',  # Canada
    'CA-ON',  # Other canadian provinces 
    'CA-AB',
    'CA-BC',
    'CA-MB',
    'CA-NB',
    'CA-NF',
    'CA-NS',
    'CA-NT',
    'CA-NU',
    'CA-PE',
    'CAZ',  # Canada - Australia - New Zealand
    'RNA',  # North America
    'US',  # United States
    'USA',  # United States
    'GLO',  # Global average 
    'RoW',  # Rest of the world
]

In [8]:
create_new_database_with_esm_results(
    mapping=mapping,
    model=model,
    esm_location='CA-QC',
    esm_results=annual_prod,
    unit_conversion=unit_conversion,
    db=db,
    locations_ranking=my_ranking,
    accepted_locations=['CA-QC', 'CA'],
    new_db_name='ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_ESM',
    technology_compositions=technology_compositions,
    tech_specifics=technology_specifics,
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC,
    tech_to_remove_layers=tech_to_remove_from_layers,
    new_end_use_types=new_end_use_types,
)

The technology DHN_RENOVATION is not in the mapping file. It cannot be considered in the result LCI dataset.
HYDRO_DAM
HYDRO_RIVER
TRAFO_MH
NUCLEAR
TRAFO_ML
TIDAL
TRAFO_HM
WIND_ONSHORE
SMR
IND_BOILER_GAS
IND_BOILER_WASTE
IND_DIRECT_ELEC
IND_HP_ELEC
DEC_HP_ELEC
HT_LT_DEC
TRAIN_FREIGHT_H2_HYBRID_LD
TRAIN_FREIGHT_H2_HYBRID_ELD
CAR_PROPANE_SD
CAR_PROPANE_MD
CAR_PROPANE_LD
BUS_PROPANE_SD
SCHOOLBUS_PROPANE_SD
COACH_PROPANE_MD
COACH_PROPANE_LD
NG_COMP_MH
NG_EXP_HM
SNG_NG
HYDRO_GAS
Vacuuming database 


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:57


Title: Writing activities to SQLite3 database:
  Started: 06/27/2024 10:52:00
  Finished: 06/27/2024 10:52:58
  Total time elapsed: 00:00:57
  CPU %: 47.20
  Memory %: 17.37
